In [1]:
import pandas as pd
import numpy as np
import csv
import math

# Read the json file from kevin

In [4]:
data_df_layer_1 = pd.read_json('../layer_1_output.json')

# Find the unique phrase in layer1

In [10]:
df_phrase_layer_1 = pd.read_json('./top_k_output/layer_1.json',index=[0])

TypeError: read_json() got an unexpected keyword argument 'index'